This notebook shall perform all the functions that the ones written for combining, cleaning, feature engineering (preliminary), geolocation, and feature engineering (final) files are designed to do for efficiency, given the knowledge of the workflow that has been established.

Import necessary packages.

In [1]:
# Apache parquet files (to save space)
import pyarrow as pa
import pyarrow.parquet as pq

# Dataframes and numerical
import pandas as pd
import numpy as np

# Geolocation
import geopandas as gpd
import matplotlib.pyplot as plt

# Increase pandas default display 
pd.options.display.max_rows = 250
pd.options.display.max_columns = 250

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

Load CSV files for months and years into their own dataframes and combine them.

In [2]:
CB_01 = pd.read_csv('CitiBike_data/Raw/New_Format/202105-citibike-tripdata.csv', low_memory=False)
CB_02 = pd.read_csv('CitiBike_data/Raw/New_Format/202106-citibike-tripdata.csv', low_memory=False)
CB_03 = pd.read_csv('CitiBike_data/Raw/New_Format/202107-citibike-tripdata.csv', low_memory=False)
CB_04 = pd.read_csv('CitiBike_data/Raw/New_Format/202108-citibike-tripdata.csv', low_memory=False)
CB_05 = pd.read_csv('CitiBike_data/Raw/New_Format/202109-citibike-tripdata.csv', low_memory=False)
CB_06 = pd.read_csv('CitiBike_data/Raw/New_Format/202110-citibike-tripdata.csv', low_memory=False)
CB_07 = pd.read_csv('CitiBike_data/Raw/New_Format/202111-citibike-tripdata.csv', low_memory=False)
CB_08 = pd.read_csv('CitiBike_data/Raw/New_Format/202112-citibike-tripdata.csv', low_memory=False)
CB_09 = pd.read_csv('CitiBike_data/Raw/New_Format/202201-citibike-tripdata.csv', low_memory=False)
CB_10 = pd.read_csv('CitiBike_data/Raw/New_Format/202202-citibike-tripdata.csv', low_memory=False)
CB_11 = pd.read_csv('CitiBike_data/Raw/New_Format/202203-citibike-tripdata.csv', low_memory=False)
CB_12 = pd.read_csv('CitiBike_data/Raw/New_Format/202204-citibike-tripdata.csv', low_memory=False)

months = [CB_01, CB_02, CB_03, CB_04, CB_05, CB_06, CB_07, CB_08, CB_09, CB_10, CB_11, CB_12]

CB_Data = pd.concat(months, ignore_index=True, sort=False)

Drop columns that are not relevant.

In [3]:
drop_col = ['ride_id', 'start_station_id', 'end_station_id']
CB_Data = CB_Data.drop(axis = 1, columns = drop_col)

Count total number of rides during that time frame (May 2021 through April 2021).

In [4]:
no_rides = len(CB_Data)

Check for null values.

In [5]:
# "Ghost bikes" coming in from unknown locations
bike_ghos = CB_Data.loc[pd.isnull(CB_Data.start_station_name)]
no_bike_ghos = len(bike_ghos)

# Bikes that are lost, i.e. not docked at the end
bike_lost = CB_Data.loc[pd.isnull(CB_Data.end_station_name)]
no_bike_lost = len(bike_lost)

# Bikes that are docked at the same station they are picked up, for joyride, rider changing mind, defective bike, etc.
bike_joyr = CB_Data.loc[CB_Data.start_station_name == CB_Data.end_station_name]
no_bike_joyr = len(bike_joyr)

print(f'Total number of Citibike rides from May 2021 through April 2022: {no_rides}')
print(f'Total number of "ghost bikes" in that time frame: {no_bike_ghos}')
print(f'Total number of lost bikes in that time frame: {no_bike_lost}')
print(f'Total number of bikes being docked at the same location in that time frame: {no_bike_joyr}')
print(f'Total number of "dud rides" to be removed from data: {no_bike_ghos + no_bike_lost + no_bike_joyr}')

print(f'Percentage of rides from May 2021 through April 2022 with bikes missing: \
      {100*(no_bike_ghos + no_bike_lost)/no_rides}')
print(f'Percentage of rides from May 2021 through April 2022 with bikes docked at the same location: \
      {100*no_bike_joyr/no_rides}')
print(f'Total number of "dud rides" to be removed from data: \
      {100*(no_bike_ghos + no_bike_lost + no_bike_joyr)/no_rides}')

Total number of Citibike rides from May 2021 through April 2022: 28816548
Total number of "ghost bikes" in that time frame: 390
Total number of lost bikes in that time frame: 103846
Total number of bikes being docked at the same location in that time frame: 1541775
Total number of "dud rides" to be removed from data: 1646011
Percentage of rides from May 2021 through April 2022 with bikes missing:       0.36172271571181946
Percentage of rides from May 2021 through April 2022 with bikes docked at the same location:       5.350311217013225
Total number of "dud rides" to be removed from data:       5.712033932725044


Now, eliminate these "dud rides" once and for all.

In [6]:
dud_rides = bike_ghos.index.tolist() + bike_lost.index.tolist() + bike_joyr.index.tolist()

CB_Data = CB_Data.drop(axis = 0, index = dud_rides)

In [7]:
print(f'Total number of rides to work with after clearning: {len(CB_Data)}')

Total number of rides to work with after clearning: 27170924


Make sure that all the values of **rideable_type** are consistent. In other words, **docked_bike** = **classic_bike**.

In [8]:
CB_Data.rideable_type.loc[CB_Data.rideable_type == 'classic_bike'] = 'Classic Bike'
CB_Data.rideable_type.loc[CB_Data.rideable_type == 'docked_bike'] = 'Classic Bike'
CB_Data.rideable_type.loc[CB_Data.rideable_type == 'electric_bike'] = 'Electric Bike'

Capitalize **member_casual** entries.

In [9]:
CB_Data.member_casual.loc[CB_Data.member_casual == 'member'] = 'Member'
CB_Data.member_casual.loc[CB_Data.member_casual == 'casual'] = 'Casual'

Convert to timestamps from strings to actual timestamp data types in order to work for extracting date and time information from them to be used as features, as well as calculating ride durations.

In [10]:
# From https://dataindependent.com/pandas/pandas-to-datetime-string-to-date-pd-to_datetime/
CB_Data.started_at = pd.to_datetime(CB_Data.started_at, format="%Y-%m-%d %H:%M:%S")
CB_Data.ended_at = pd.to_datetime(CB_Data.ended_at, format="%Y-%m-%d %H:%M:%S")

Since the heavy lifting associated with creating a master file of all stations in the CitiBike system have already been completed, just import it from its .parquet file at this point.

In [11]:
CB_Stations = pq.read_table('CitiBike_data/202206-citibike-stations.parquet').to_pandas()

In order to normalize all coordinates of the distinct stations as well as the travel distances, durations, and speeds between the various stations; the averaged latitudes and longitudes of the dataframe **CB_Stations** shall replace those provided in the **CB_Data** one in addition to assigning borough and neighboorhood associations.

In [12]:
# Join CB_Data with CB_Stations
# Replace old coordinates with new ones
CB_Data = CB_Data.join(CB_Stations, on='start_station_name', how='right')
CB_Data = CB_Data.drop(columns=['start_lat', 'start_lng'])
CB_Data = CB_Data.rename(columns={'lat': 'start_lat', 'lng': 'start_lng', 'boro': 'start_boro', 'hood': 'start_hood'})
CB_Data = CB_Data.join(CB_Stations, on='end_station_name', how='right')
CB_Data = CB_Data.drop(columns=['end_lat', 'end_lng'])
CB_Data = CB_Data.rename(columns={'lat': 'end_lat', 'lng': 'end_lng', 'boro': 'end_boro', 'hood': 'end_hood'})

Sort by index values.

In [13]:
CB_Data = CB_Data.sort_index(ascending=True)

Drop any **NaN** values in the new dataframe.

In [14]:
CB_Data = CB_Data.dropna(axis=0)

Reconvert index in dataframe to integer.

In [15]:
CB_Data.index = CB_Data.index.astype('int64')

Calculate (Manhattan) distance and speed with normalized coordinates.

In [16]:
# Conversion factor here:
# https://www.usgs.gov/faqs/how-much-distance-does-degree-minute-and-second-cover-your-maps#:~:text=One%20degree%20of%20latitude%20equals,one%20second%20equals%2080%20feet.
CB_Data['distance_mi'] = 69 * ( abs( CB_Data.start_lat - CB_Data.end_lat ) 
                                  + abs( CB_Data.start_lng - CB_Data.end_lng ) )
CB_Data['duration_min'] = (CB_Data.ended_at - CB_Data.started_at)/np.timedelta64(1,'m')
CB_Data['speed_mph'] = CB_Data.distance_mi / (CB_Data.duration_min / 60)

With the lengthy computations completed, dummify various values based on the timestamp.

In [17]:
CB_Data['year'] = CB_Data.started_at.dt.year
CB_Data['month'] = CB_Data.started_at.dt.month
CB_Data['week_of_year'] = CB_Data.started_at.dt.week
CB_Data['day_of_week'] = CB_Data.started_at.dt.day_of_week
CB_Data['hour_of_day'] = CB_Data.started_at.dt.hour

Ensure that nominal dummy values are integers.

In [18]:
CB_Data = CB_Data.astype({'year': int, 'month': int, 'week_of_year': int,
                                  'day_of_week': int, 'hour_of_day': int})

Reindex columns for readability.

In [19]:
col_names = ['member_casual', 'rideable_type',
             'started_at', 'start_station_name', 'start_lat', 'start_lng', 'start_boro', 'start_hood',
             'ended_at', 'end_station_name', 'end_lat', 'end_lng', 'end_boro', 'end_hood',
             'year', 'month', 'week_of_year', 'day_of_week', 'hour_of_day',
             'duration_min', 'distance_mi', 'speed_mph']
CB_Data = CB_Data.reindex(columns=col_names)

Perform a final check of the structure and integrity of the dataframe before finally writing it to a .parquet file.

In [20]:
CB_Data.head()

,member_casual,rideable_type,started_at,start_station_name,start_lat,start_lng,start_boro,start_hood,ended_at,end_station_name,end_lat,end_lng,end_boro,end_hood,year,month,week_of_year,day_of_week,hour_of_day,duration_min,distance_mi,speed_mph
1,Member,Classic Bike,2021-05-13 12:48:08,Broadway & W 25 St,40.742868,-73.989186,Manhattan,Flatiron District,2021-05-13 13:07:37,E 2 St & Avenue B,40.722175,-73.983688,Manhattan,East Village,2021,5,19,3,12,19.483333,1.807254,5.565537
2,Member,Classic Bike,2021-05-16 08:30:13,46 Ave & 5 St,40.747310,-73.954510,Queens,Long Island City,2021-05-16 08:45:47,34th Ave & Vernon Blvd,40.765354,-73.939863,Queens,Long Island City,2021,5,19,6,8,15.566667,2.255646,8.694139
3,Member,Classic Bike,2021-05-01 08:38:14,46 Ave & 5 St,40.747310,-73.954510,Queens,Long Island City,2021-05-01 08:54:27,34th Ave & Vernon Blvd,40.765354,-73.939863,Queens,Long Island City,2021,5,17,5,8,16.216667,2.255646,8.345659
4,Member,Classic Bike,2021-05-09 08:12:31,46 Ave & 5 St,40.747310,-73.954510,Queens,Long Island City,2021-05-09 08:27:05,34th Ave & Vernon Blvd,40.765354,-73.939863,Queens,Long Island City,2021,5,18,6,8,14.566667,2.255646,9.290991
5,Member,Classic Bike,2021-05-27 07:52:27,E 123 St & Lexington Ave,40.802926,-73.937900,Manhattan,East Harlem,2021-05-27 08:09:01,1 Ave & E 78 St,40.771404,-73.953516,Manhattan,Upper East Side,2021,5,21,3,7,16.566667,3.252584,11.779982


In [21]:
CB_Data.tail()

,member_casual,rideable_type,started_at,start_station_name,start_lat,start_lng,start_boro,start_hood,ended_at,end_station_name,end_lat,end_lng,end_boro,end_hood,year,month,week_of_year,day_of_week,hour_of_day,duration_min,distance_mi,speed_mph
28816543,Member,Classic Bike,2022-04-22 13:08:15,Allen St & Rivington St,40.720196,-73.989978,Manhattan,Lower East Side,2022-04-22 13:29:31,W 29 St & 9 Ave,40.750073,-73.998393,Manhattan,Chelsea,2022,4,16,4,13,21.266667,2.642086,7.454162
28816544,Member,Classic Bike,2022-04-13 17:10:49,St. Nicholas Terrace & Convent Ave,40.821292,-73.947867,Manhattan,Harlem,2022-04-13 17:26:08,E 138 St & Grand Concourse,40.813224,-73.930605,Bronx,Mott Haven,2022,4,15,2,17,15.316667,1.747766,6.846525
28816545,Member,Classic Bike,2022-04-12 15:58:47,E 88 St & 1 Ave,40.778301,-73.948813,Manhattan,Upper East Side,2022-04-12 16:41:17,E 91 St & 2 Ave,40.781151,-73.949634,Manhattan,Upper East Side,2022,4,15,1,15,42.500000,0.253228,0.357498
28816546,Member,Electric Bike,2022-04-28 15:52:49,W 55 St & 6 Ave,40.763189,-73.978435,Manhattan,Midtown,2022-04-28 16:06:31,E 91 St & 2 Ave,40.781151,-73.949634,Manhattan,Upper East Side,2022,4,17,3,15,13.700000,3.226664,14.131377
28816547,Member,Electric Bike,2022-04-11 19:55:31,W 87 St & West End Ave,40.789625,-73.977567,Manhattan,Upper West Side,2022-04-11 20:09:19,E 91 St & 2 Ave,40.781151,-73.949634,Manhattan,Upper East Side,2022,4,15,0,19,13.800000,2.512130,10.922306


In [22]:
CB_Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27143436 entries, 1 to 28816547
Data columns (total 22 columns):
 #   Column              Dtype         
---  ------              -----         
 0   member_casual       object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   start_station_name  object        
 4   start_lat           float64       
 5   start_lng           float64       
 6   start_boro          object        
 7   start_hood          object        
 8   ended_at            datetime64[ns]
 9   end_station_name    object        
 10  end_lat             float64       
 11  end_lng             float64       
 12  end_boro            object        
 13  end_hood            object        
 14  year                int64         
 15  month               int64         
 16  week_of_year        int64         
 17  day_of_week         int64         
 18  hour_of_day         int64         
 19  duration_min        float64       
 20  

In [23]:
CB_Data.dtypes

member_casual                 object
rideable_type                 object
started_at            datetime64[ns]
start_station_name            object
start_lat                    float64
start_lng                    float64
start_boro                    object
start_hood                    object
ended_at              datetime64[ns]
end_station_name              object
end_lat                      float64
end_lng                      float64
end_boro                      object
end_hood                      object
year                           int64
month                          int64
week_of_year                   int64
day_of_week                    int64
hour_of_day                    int64
duration_min                 float64
distance_mi                  float64
speed_mph                    float64
dtype: object

Check for null values.

In [24]:
CB_Data.isnull().sum()

member_casual         0
rideable_type         0
started_at            0
start_station_name    0
start_lat             0
start_lng             0
start_boro            0
start_hood            0
ended_at              0
end_station_name      0
end_lat               0
end_lng               0
end_boro              0
end_hood              0
year                  0
month                 0
week_of_year          0
day_of_week           0
hour_of_day           0
duration_min          0
distance_mi           0
speed_mph             0
dtype: int64

No null values and the dataframe has all the values that are needed for the EDA. Now it is time to export this completed dataframe to a .parquet file.

In [25]:
CB_Data_arrow = pa.Table.from_pandas(CB_Data)
pq.write_table(CB_Data_arrow, 'CitiBike_data/202105-202204-citibike-tripdata.parquet')